# 0.Setting

- google Colab 환경에서 진행하였습니다.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

In [ ]:
!pip install magenta==2.1.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.4 MB 13.8 MB/s 
     |████████████████████████████████| 69 kB 8.0 MB/s 
     |████████████████████████████████| 204 kB 90.9 MB/s 
     |████████████████████████████████| 3.6 MB 71.8 MB/s 
     |████████████████████████████████| 209 kB 81.9 MB/s 
     |████████████████████████████████| 90 kB 10.8 MB/s 
     |████████████████████████████████| 1.4 MB 71.1 MB/s 
     |████████████████████████████████| 352 kB 82.8 MB/s 
     |████████████████████████████████| 10.9 MB 71.5 MB/s 
     |████████████████████████████████| 254 kB 87.6 MB/s 
     |████████████████████████████████| 5.6 MB 28.6 MB/s 
     |████████████████████████████████| 2.3 MB 74.9 MB/s 
     |████████████████████████████████| 508 kB 74.3 MB/s 
     |████████████████████████████████| 2.3 MB 64.2 MB/s 
     |████████████████████████████████| 272 kB 88.4 MB/s 
     |███████████████████████████

In [ ]:
import magenta

# 1.Preprocessing

- mid -> note_seq -> tfrecord

In [ ]:
data_root= '/content/drive/MyDrive/MusicVAE/groove'
csv_file = data_root+'/info.csv'

In [ ]:
tfrec_root = '/content/drive/MyDrive/MusicVAE/tfrec/example.tfrecord'

In [ ]:
from magenta.scripts.convert_dir_to_note_sequences import convert_directory

In [ ]:
convert_directory(data_root,tfrec_root,recursive=True)
# .mid -> note_seq -> .tfrecord
# tfrec_root 위치에 .mid파일을 .tfrecord로 변환하여 학습이 가능한 형태로 저장해줌

INFO:tensorflow:Converting files in '/content/drive/MyDrive/MusicVAE/groove/'.
INFO:tensorflow:0 files converted.
INFO:tensorflow:Converting files in '/content/drive/MyDrive/MusicVAE/groove/drummer4'.
INFO:tensorflow:Converting files in '/content/drive/MyDrive/MusicVAE/groove/drummer4/session1'.
INFO:tensorflow:Converted MIDI file /content/drive/MyDrive/MusicVAE/groove/drummer4/session1/1_rock_87_beat_4-4.mid.
INFO:tensorflow:Converted MIDI file /content/drive/MyDrive/MusicVAE/groove/drummer4/session1/2_jazz_158_beat_4-4.mid.
INFO:tensorflow:Converted MIDI file /content/drive/MyDrive/MusicVAE/groove/drummer4/session1/9_latin-merengue_158_beat_4-4.mid.
INFO:tensorflow:Converted MIDI file /content/drive/MyDrive/MusicVAE/groove/drummer4/session1/5_latin-brazilian_184_beat_4-4.mid.
INFO:tensorflow:Converted MIDI file /content/drive/MyDrive/MusicVAE/groove/drummer4/session1/4_latin-brazilian_92_beat_4-4.mid.
INFO:tensorflow:Converted MIDI file /content/drive/MyDrive/MusicVAE/groove/drummer4

## Config

- 학습을 위해 모델을 포함한 Config 정의
- train_examples_path 변수에 학습용으로 변환한 .tfrecord의 path를 넣어줌
- Config 설정 참고: https://github.com/magenta/magenta/blob/main/magenta/models/music_vae/configs.py

In [ ]:
import collections

class Config(collections.namedtuple(
    'Config',
    ['model', 'hparams', 'note_sequence_augmenter', 'data_converter',
     'train_examples_path', 'eval_examples_path', 'tfds_name'])):

  def values(self):
    return self._asdict()

Config.__new__.__defaults__ = (None,) * len(Config._fields)


def update_config(config, update_dict):
  config_dict = config.values()
  config_dict.update(update_dict)
  return Config(**config_dict)


CONFIG_MAP = {}

In [ ]:
from magenta.common import merge_hparams
from magenta.contrib import training as contrib_training
from magenta.models.music_vae import MusicVAE
from magenta.models.music_vae import lstm_models
from magenta.models.music_vae import data

HParams = contrib_training.HParams

# GrooVAE configs
CONFIG_MAP['groovae_4bar'] = Config(
    model=MusicVAE(lstm_models.BidirectionalLstmEncoder(),
                   lstm_models.GrooveLstmDecoder()), # MusicVAE 정의
    hparams=merge_hparams(
        lstm_models.get_default_hparams(),
        HParams(
            batch_size=512,
            max_seq_len=16 * 4,  # 4 bars w/ 16 steps per bar
            z_size=256,
            enc_rnn_size=[512],
            dec_rnn_size=[256, 256],
            max_beta=0.2,
            free_bits=48,
            dropout_keep_prob=0.3,
        )),
    note_sequence_augmenter=None,
    data_converter=data.GrooveConverter(
        split_bars=4, steps_per_quarter=4, quarters_per_bar=4,
        max_tensors_per_notesequence=20,
        pitch_classes=data.ROLAND_DRUM_PITCH_CLASSES,
        inference_pitch_classes=data.REDUCED_DRUM_PITCH_CLASSES),
    # tfds_name='groove/4bar-midionly',
    train_examples_path='/content/drive/MyDrive/MusicVAE/tfrec/example.tfrecord',
)

### 인코더: BidirectionalLSTM

- 단방향으로만 정보를 전달하던 LSTM을 개선한 양방향 LSTM
- longer-term context 정보 전달에 좀 더 강점이 있다.

### 디코더: GrooveLSTM

- 베르누이분포를 기반으로 샘플링을 진행한다.

## Train Function

In [ ]:
# Copyright 2022 The Magenta Authors.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

"""MusicVAE training script."""
import os

from magenta.models.music_vae import configs
from magenta.models.music_vae import data
import tensorflow.compat.v1 as tf
import tf_slim

# Should not be called from within the graph to avoid redundant summaries.
def _trial_summary(hparams, examples_path, output_dir):
  """Writes a tensorboard text summary of the trial."""

  examples_path_summary = tf.summary.text(
      'examples_path', tf.constant(examples_path, name='examples_path'),
      collections=[])

  hparams_dict = hparams.values()

  # Create a markdown table from hparams.
  header = '| Key | Value |\n| :--- | :--- |\n'
  keys = sorted(hparams_dict.keys())
  lines = ['| %s | %s |' % (key, str(hparams_dict[key])) for key in keys]
  hparams_table = header + '\n'.join(lines) + '\n'

  hparam_summary = tf.summary.text(
      'hparams', tf.constant(hparams_table, name='hparams'), collections=[])

  with tf.Session() as sess:
    writer = tf.summary.FileWriter(output_dir, graph=sess.graph)
    writer.add_summary(examples_path_summary.eval())
    writer.add_summary(hparam_summary.eval())
    writer.close()


def _get_input_tensors(dataset, config):
  """Get input tensors from dataset."""
  batch_size = config.hparams.batch_size
  iterator = tf.data.make_one_shot_iterator(dataset)
  (input_sequence, output_sequence, control_sequence,
   sequence_length) = iterator.get_next()
  input_sequence.set_shape(
      [batch_size, None, config.data_converter.input_depth])
  output_sequence.set_shape(
      [batch_size, None, config.data_converter.output_depth])
  if not config.data_converter.control_depth:
    control_sequence = None
  else:
    control_sequence.set_shape(
        [batch_size, None, config.data_converter.control_depth])
  sequence_length.set_shape([batch_size] + sequence_length.shape[1:].as_list())

  return {
      'input_sequence': input_sequence,
      'output_sequence': output_sequence,
      'control_sequence': control_sequence,
      'sequence_length': sequence_length
  }


def train(train_dir,
          config,
          dataset_fn,
          checkpoints_to_keep=5,
          keep_checkpoint_every_n_hours=1,
          num_steps=None,
          master='',
          num_sync_workers=0,
          num_ps_tasks=0,
          task=0):
  """Train loop."""
  tf.gfile.MakeDirs(train_dir)
  is_chief = (task == 0)
  # if is_chief:
  #   _trial_summary(
  #       config.hparams, config.train_examples_path or config.tfds_name,
  #       train_dir)

  with tf.Graph().as_default():
    with tf.device(tf.train.replica_device_setter(
        num_ps_tasks, merge_devices=True)):

      model = config.model
      model.build(config.hparams,
                  config.data_converter.output_depth,
                  is_training=True)

      optimizer = model.train(**_get_input_tensors(dataset_fn(), config))

      hooks = []
      if num_sync_workers:
        optimizer = tf.train.SyncReplicasOptimizer(
            optimizer,
            num_sync_workers)
        hooks.append(optimizer.make_session_run_hook(is_chief))

      grads, var_list = list(zip(*optimizer.compute_gradients(model.loss)))
      global_norm = tf.global_norm(grads)
      tf.summary.scalar('global_norm', global_norm)

      if config.hparams.clip_mode == 'value':
        g = config.hparams.grad_clip
        clipped_grads = [tf.clip_by_value(grad, -g, g) for grad in grads]
      elif config.hparams.clip_mode == 'global_norm':
        clipped_grads = tf.cond(
            global_norm < config.hparams.grad_norm_clip_to_zero,
            lambda: tf.clip_by_global_norm(  # pylint:disable=g-long-lambda
                grads, config.hparams.grad_clip, use_norm=global_norm)[0],
            lambda: [tf.zeros(tf.shape(g)) for g in grads])
      else:
        raise ValueError(
            'Unknown clip_mode: {}'.format(config.hparams.clip_mode))
      train_op = optimizer.apply_gradients(
          list(zip(clipped_grads, var_list)),
          global_step=model.global_step,
          name='train_step')

      logging_dict = {'global_step': model.global_step,
                      'loss': model.loss}

      hooks.append(tf.train.LoggingTensorHook(logging_dict, every_n_iter=100))
      if num_steps:
        hooks.append(tf.train.StopAtStepHook(last_step=num_steps))

      scaffold = tf.train.Scaffold(
          saver=tf.train.Saver(
              max_to_keep=checkpoints_to_keep,
              keep_checkpoint_every_n_hours=keep_checkpoint_every_n_hours))
      tf_slim.training.train(
          train_op=train_op,
          logdir=train_dir,
          scaffold=scaffold,
          hooks=hooks,
          save_checkpoint_secs=60,
          master=master,
          is_chief=is_chief)


def run(config_map,
        tf_file_reader=tf.data.TFRecordDataset,
        file_reader=tf.python_io.tf_record_iterator,
        is_training=True):
  """Load model params, save config file and start trainer.

  Args:
    config_map: Dictionary mapping configuration name to Config object.
    tf_file_reader: The tf.data.Dataset class to use for reading files.
    file_reader: The Python reader to use for reading files.

  Raises:
    ValueError: if required flags are missing or invalid.
  """
  config = config_map['groovae_4bar']
  train_dir = '/content/drive/MyDrive/MusicVAE/train'
  num_steps = None # epoch

  def dataset_fn():
    return data.get_dataset(
        config,
        tf_file_reader=tf_file_reader,
        is_training=True,
        cache_dataset=True)

  if is_training == True:
    train(
        train_dir,
        config=config,
        dataset_fn=dataset_fn,
        num_steps=num_steps)
        # checkpoints_to_keep=FLAGS.checkpoints_to_keep,
        # keep_checkpoint_every_n_hours=FLAGS.keep_checkpoint_every_n_hours,
        # num_steps=FLAGS.num_steps,
        # master=FLAGS.master,
        # num_sync_workers=FLAGS.num_sync_workers,
        # num_ps_tasks=FLAGS.num_ps_tasks,
        # task=FLAGS.task)
  else:
    print("EVAL")
    # num_batches = FLAGS.eval_num_batches or data.count_examples(
    #     config.eval_examples_path,
    #     config.tfds_name,
    #     config.data_converter,
    #     file_reader) // config.hparams.batch_size
    # eval_dir = os.path.join(run_dir, 'eval' + FLAGS.eval_dir_suffix)
    # evaluate(
    #     train_dir,
    #     eval_dir,
    #     config=config,
    #     dataset_fn=dataset_fn,
    #     num_batches=num_batches,
    #     master=FLAGS.master)


# def main(unused_argv):
#   tf.logging.set_verbosity(FLAGS.log)
#   run(configs.CONFIG_MAP)


# def console_entry_point():
#   tf.disable_v2_behavior()
#   tf.app.run(main)


# if __name__ == '__main__':
#   console_entry_point()

# 2. Start Train

In [ ]:
#@title 기본 제목 텍스트
run(CONFIG_MAP)

INFO:tensorflow:Building MusicVAE model with BidirectionalLstmEncoder, GrooveLstmDecoder, and hparams:
{'max_seq_len': 64, 'z_size': 256, 'free_bits': 48, 'max_beta': 0.2, 'beta_rate': 0.0, 'batch_size': 512, 'grad_clip': 1.0, 'clip_mode': 'global_norm', 'grad_norm_clip_to_zero': 10000, 'learning_rate': 0.001, 'decay_rate': 0.9999, 'min_learning_rate': 1e-05, 'conditional': True, 'dec_rnn_size': [256, 256], 'enc_rnn_size': [512], 'dropout_keep_prob': 0.3, 'sampling_schedule': 'constant', 'sampling_rate': 0.0, 'use_cudnn': False, 'residual_encoder': False, 'residual_decoder': False, 'control_preprocessing_rnn_size': [256]}
INFO:tensorflow:
Encoder Cells (bidirectional):
  units: [512]

INFO:tensorflow:
Decoder Cells:
  units: [256, 256]

INFO:tensorflow:Reading examples from file: /content/drive/MyDrive/MusicVAE/tfrec/example.tfrecord


/usr/local/lib/python3.7/dist-packages/magenta/contrib/rnn.py:751: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  self._names["W"], [input_size + self._num_units, self._num_units * 4])
/usr/local/lib/python3.7/dist-packages/magenta/contrib/rnn.py:754: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  initializer=tf.constant_initializer(0.0))
/usr/local/lib/python3.7/dist-packages/magenta/models/music_vae/base_model.py:199: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  kernel_initializer=tf.random_normal_initializer(stddev=0.001))
/usr/local/lib/python3.7/dist-packages/keras/legacy_tf_layers/core.py:261: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into /content/drive/MyDrive/MusicVAE/train/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:global_step = 0, loss = 567.23566
INFO:tensorflow:global_step/sec: 3.5004
INFO:tensorflow:global_step = 100, loss = 206.80963 (28.571 sec)
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 130...
INFO:tensorflow:Saving checkpoints for 130 into /content/drive/MyDrive/MusicVAE/train/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 130...
INFO:tensorflow:global_step/sec: 3.98368
INFO:tensorflow:global_step = 200, loss = 188.05899 (25.102 sec)
INFO:tensorflow:global_step/sec: 4.15808
INFO:tensorflow:global_step = 300, los

KeyboardInterrupt: ignored

# 3. Generate sequence

In [ ]:
# GrooVAE configs
CONFIG_MAP['groovae_4bar'] = Config(
    model=MusicVAE(lstm_models.BidirectionalLstmEncoder(),
                   lstm_models.GrooveLstmDecoder()), # MusicVAE 정의
    hparams=merge_hparams(
        lstm_models.get_default_hparams(),
        HParams(
            batch_size=512,
            max_seq_len=16 * 4,  # 4 bars w/ 16 steps per bar
            z_size=256,
            enc_rnn_size=[512],
            dec_rnn_size=[256, 256],
            max_beta=0.2,
            free_bits=48,
            dropout_keep_prob=0.3,
        )),
    note_sequence_augmenter=None,
    data_converter=data.GrooveConverter(
        split_bars=4, steps_per_quarter=4, quarters_per_bar=4,
        max_tensors_per_notesequence=20,
        pitch_classes=data.ROLAND_DRUM_PITCH_CLASSES,
        inference_pitch_classes=data.REDUCED_DRUM_PITCH_CLASSES),
    # tfds_name='groove/4bar-midionly',
    train_examples_path='/content/drive/MyDrive/MusicVAE/tfrec/example.tfrecord',
)

In [ ]:
from magenta.models.music_vae.trained_model import TrainedModel

In [ ]:
model = TrainedModel(
    config=CONFIG_MAP['groovae_4bar'],
    batch_size=1,
    checkpoint_dir_or_path='/content/drive/MyDrive/MusicVAE/train')

INFO:tensorflow:Building MusicVAE model with BidirectionalLstmEncoder, GrooveLstmDecoder, and hparams:
{'max_seq_len': 64, 'z_size': 256, 'free_bits': 48, 'max_beta': 0.2, 'beta_rate': 0.0, 'batch_size': 1, 'grad_clip': 1.0, 'clip_mode': 'global_norm', 'grad_norm_clip_to_zero': 10000, 'learning_rate': 0.001, 'decay_rate': 0.9999, 'min_learning_rate': 1e-05, 'conditional': True, 'dec_rnn_size': [256, 256], 'enc_rnn_size': [512], 'dropout_keep_prob': 0.3, 'sampling_schedule': 'constant', 'sampling_rate': 0.0, 'use_cudnn': False, 'residual_encoder': False, 'residual_decoder': False, 'control_preprocessing_rnn_size': [256]}
INFO:tensorflow:
Encoder Cells (bidirectional):
  units: [512]

INFO:tensorflow:
Decoder Cells:
  units: [256, 256]



/usr/local/lib/python3.7/dist-packages/magenta/models/music_vae/lstm_utils.py:99: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  name=name),
/usr/local/lib/python3.7/dist-packages/keras/legacy_tf_layers/core.py:261: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  return layer.apply(inputs)
/usr/local/lib/python3.7/dist-packages/magenta/contrib/rnn.py:751: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  self._names["W"], [input_size + self._num_units, self._num_units * 4])
/usr/local/lib/python3.7/dist-packages/magenta/contrib/rnn.py:754: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  initializer=tf.constant_initializer(0.0))


INFO:tensorflow:Restoring parameters from /content/drive/MyDrive/MusicVAE/train/model.ckpt-20084


/usr/local/lib/python3.7/dist-packages/magenta/models/music_vae/base_model.py:199: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  kernel_initializer=tf.random_normal_initializer(stddev=0.001))
/usr/local/lib/python3.7/dist-packages/magenta/models/music_vae/base_model.py:205: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  kernel_initializer=tf.random_normal_initializer(stddev=0.001))


In [ ]:
generated_sequence = model.sample(n=1, length=16*4, temperature=0.5)

In [ ]:
generated_sequence[0]

tempos {
  qpm: 120.0
}
notes {
  pitch: 36
  velocity: 78
  start_time: -0.003606407204642892
  end_time: 0.12139359279535711
  instrument: 9
  is_drum: true
}
notes {
  pitch: 42
  velocity: 55
  start_time: 0.0010058233747258782
  end_time: 0.12600582337472588
  instrument: 9
  is_drum: true
}
notes {
  pitch: 51
  velocity: 107
  start_time: 0.008768817409873009
  end_time: 0.133768817409873
  instrument: 9
  is_drum: true
}
notes {
  pitch: 51
  velocity: 57
  start_time: 0.14689947664737701
  end_time: 0.271899476647377
  instrument: 9
  is_drum: true
}
notes {
  pitch: 42
  velocity: 10
  start_time: 0.2489376040175557
  end_time: 0.3739376040175557
  instrument: 9
  is_drum: true
}
notes {
  pitch: 51
  velocity: 54
  start_time: 0.25872526690363884
  end_time: 0.38372526690363884
  instrument: 9
  is_drum: true
}
notes {
  pitch: 36
  velocity: 14
  start_time: 0.3732155819889158
  end_time: 0.4982155819889158
  instrument: 9
  is_drum: true
}
notes {
  pitch: 51
  velocity: 5

In [ ]:
import note_seq

In [ ]:
note_seq.sequence_proto_to_midi_file(generated_sequence[0], 'sample.mid')